In [1]:
# -*- coding: utf-8 -*-

#Convert mutliclasses into Single Class

#Resolution: 0.004

#Created on Mon Apr 11 16:42:38 2022

#@author: li.chao.987@s.kyushu-u.ac.jp


from osgeo import gdal
import numpy as np
import os

In [2]:
aimFolder = "F:\\17_Article\\01_Data\\01_LandCover\\LandCover0004WGS\\"
#os.mkdir("F:\\17_Article\\01_Data\\01_LandCover\\LandCover0004WGS")

year=2001
while year < 2021:
    original_raster = gdal.Open("F:\\17_Article\\01_Data\\01_LandCover\\LandCoverMerge\\LandCover_UMD_" + str(year) + "_500m.tif")
    #print(original_raster)

    raster_rprj = gdal.Warp(aimFolder + "LandCover_UMD_"+ str(year) +"_0004_WGS.tif", original_raster, dstSRS = "EPSG:4326", xRes = 0.004, yRes = 0.004, resampleAlg = "mode")
    print(raster_rprj)
    #raster_rprj = None

    original_raster = None

    band = raster_rprj.GetRasterBand(1)

    multiLista = band.ReadAsArray()

    # reclassification
    singleRasterLocation = "F:/17_Article/01_Data/01_LandCover/LandCoverSingleClass/"
    driver = gdal.GetDriverByName("GTiff")
    aim = 0
    while aim < 16:
        print(aim)
        originalToSingleLista = multiLista.copy()
        orignialToSingleListaClip = originalToSingleLista[:,37750:41750].copy()
        #orignialToSingleListaClip = 
        orignialToSingleListaClip[orignialToSingleListaClip == aim] = 100
        orignialToSingleListaClip[orignialToSingleListaClip != 100] = 0
        
        #for j in range(raster_rprj.RasterXSize):
        #    for i in range(raster_rprj.RasterYSize):
        #        if originalToSingleLista[i, j] == aim:
        #            originalToSingleLista[i, j] = 1
        #        else:
        #            originalToSingleLista[i, j] = 0
                    
        # create new file
        name = "year_" + str(year) + "_class_" + str(aim) + ".tif"
        singleRaster = driver.Create(singleRasterLocation + name, 8000, 7500, 1)
        singleRaster.GetRasterBand(1).WriteArray(orignialToSingleListaClip)
        
        proj = raster_rprj.GetProjection()
        georef = (122.0, 0.004, 0.0, 50.0, 0.0, -0.004)
        singleRaster.SetProjection(proj)
        singleRaster.SetGeoTransform(georef)
        singleRaster.FlushCache()
        singleRaster = None
        aim += 1
        
    raster_rprj = None   

    os.remove(aimFolder + "LandCover_UMD_" + str(year) + "_0004_WGS.tif")

    year = year + 1

<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000001F55B9B77B0> >
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000001F55B9B7B40> >
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000001F55B9B7870> >
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000001F55B9B7930> >
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000001F55B9B78D0> >
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000001F55B9B7E70> >
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
<osgeo.gdal.Dataset; proxy of <Swig Object of type 'GDALDatasetShadow *' at 0x000001F55B9B7810> >
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
<osgeo.gdal.Dataset; proxy of <Swig Object of ty

In [ ]:
import pandas as pd
import geopandas as gpd
import numpy as np
import rasterio
from scipy.spatial import cKDTree

# make the raster -180 -60 180 90
nx = 8000                                       # number of cells in the x direction
ny = 7500                                     # number of cells in the y direction
xmin = 122.002                                  # x coordinate of lower, left cell center 
ymin = 30.002                                     # y coordinate of lower, left cell center 
xsize = 0.004                                   # extent of cells in x direction
ysize = 0.004                                   # extent of cells in y direction
epsg = 4326                                     # get projection

def addCoord(nx,xmin,xsize,ny,ymin,ysize, epsg): # Michael Pyrcz, March, 2018                      
  # makes a 2D dataframe with coordinates based on GSLIB specification
  coords = np.zeros([nx*ny, 3])
  ixy = 0
  for iy in range(nx):
    for ix in range(ny):
      coords[ixy,1] = xmin + ix*xsize  
      coords[ixy,2] = ymin + iy*ysize 
      coords[ixy,0] = ixy + 1
      ixy = ixy + 1
      
  coords = pd.DataFrame(coords, columns = ["order", "X", "Y"])
  gdf = gpd.GeoDataFrame(coords, geometry = gpd.points_from_xy(coords.X, coords.Y))
  gdf = gdf.set_crs(epsg = epsg)
  return (gdf)

def ckdnearest(gdA, gdB, column_name):

    nA = np.array(list(gdA.geometry.apply(lambda x: (x.x, x.y))))
    nB = np.array(list(gdB.geometry.apply(lambda x: (x.x, x.y))))
    btree = cKDTree(nB)
    dist, idx = btree.query(nA, k=1)
    gdB_nearest = gdB.iloc[idx].drop(columns="geometry").reset_index(drop=True)
    gdf = pd.concat(
        [
            gdA.reset_index(drop=True),
            gdB_nearest,
            pd.Series(dist, name=column_name)
        ], 
        axis=1)

    return gdf

coords_distance_raster = addCoord(nx,xmin,xsize,ny,ymin,ysize, epsg)

Signature: os.mkdir(path, mode=511, *, dir_fd=None)
Docstring:
Create a directory.

If dir_fd is not None, it should be a file descriptor open to a directory,
  and path should be relative; path will then be relative to that directory.
dir_fd may not be implemented on your platform.
  If it is unavailable, using it will raise a NotImplementedError.

The mode argument is ignored on Windows.
Type:      builtin_function_or_method
